In [1]:
from pathlib import Path
from e4e.align import xy_auto_align, t_align
from tqdm.notebook import tqdm

In [2]:
deployment_root_path = Path('G:\\Shared drives\\E4E_FishSense\\Test Data\\2022-05 Reef Deployment\\usa_florida')
target_path = Path('Z:\\data\\2022-05 Reef Deployment outputs')

In [3]:
output_dirs = ['_'.join(bag_file.relative_to(deployment_root_path).with_suffix('').parts) for bag_file in deployment_root_path.glob('**\\*.bag')]
label_dirs = ['_'.join(bag_file.relative_to(deployment_root_path).with_suffix('').parts) + '_label' for bag_file in deployment_root_path.glob('**\\*.bag')]

In [4]:
for dir in tqdm(output_dirs):
    target_path.joinpath(dir).mkdir(parents=True, exist_ok=True)
for dir in tqdm(label_dirs):
    target_path.joinpath(dir).mkdir(parents=True, exist_ok=True)

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

In [5]:
successful_files = []
for idx, bag_file in tqdm(enumerate(deployment_root_path.glob('**\\*.bag'))):
    output_dir = Path(output_dirs[idx])
    label_dir = Path(label_dirs[idx])
    print(bag_file.as_posix())
    try:
        xy_auto_align(bag_file=bag_file, output_dir=output_dir)
        t_align(output_dir=output_dir, input_dir=output_dir, label_dir=label_dir)
    except:
        continue
    print("done")
    successful_files.append(bag_file)

0it [00:00, ?it/s]

G:/Shared drives/E4E_FishSense/Test Data/2022-05 Reef Deployment/usa_florida/2022-05-20/carysfort/FS003/260.bag


  0%|          | 0/5.750603 [00:00<?, ?it/s]


G:/Shared drives/E4E_FishSense/Test Data/2022-05 Reef Deployment/usa_florida/2022-05-20/carysfort/FS003/270.bag


  0%|          | 0/3809.987421 [00:00<?, ?it/s]


...

  0%|          | 0/20.994346 [00:00<?, ?it/s]


G:/Shared drives/E4E_FishSense/Test Data/2022-05 Reef Deployment/usa_florida/2022-05-20/carysfort/fs002/200.bag


  0%|          | 0/162.184757 [00:00<?, ?it/s]


G:/Shared drives/E4E_FishSense/Test Data/2022-05 Reef Deployment/usa_florida/2022-05-20/carysfort/fs002/340.bag
G:/Shared drives/E4E_FishSense/Test Data/2022-05 Reef Deployment/usa_florida/2022-05-20/carysfort/fs002/350.bag


  0%|          | 0/2.279053 [00:00<?, ?it/s]